## Import and define global functions

In [49]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')


In [50]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/startup_funding.csv')

In [51]:
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


## Clean Data

In [52]:
df.set_index('Sr No', inplace=True)

In [53]:
df['date_date'] = pd.to_datetime(df['Date dd/mm/yyyy'], format='%d/%m/%Y', errors='coerce')

In [54]:
df.drop('Date dd/mm/yyyy', axis=1, inplace=True)

In [55]:
df.Remarks.fillna('No Remarks', inplace=True)

In [56]:
df.columns

Index(['Startup Name', 'Industry Vertical', 'SubVertical', 'City  Location',
       'Investors Name', 'InvestmentnType', 'Amount in USD', 'Remarks',
       'date_date'],
      dtype='object')

In [57]:
df.rename(columns={
    "Startup Name": "startup_name",
    "Industry Vertical": "industry",
    "SubVertical": "subvertical",
    "City  Location": "city",
    "Investors Name": "investors",
    "Amount in USD": "amount_usd",
    "Remarks": "remarks"
    },inplace=True)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3044 entries, 1 to 3044
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   startup_name     3044 non-null   object        
 1   industry         2873 non-null   object        
 2   subvertical      2108 non-null   object        
 3   city             2864 non-null   object        
 4   investors        3020 non-null   object        
 5   InvestmentnType  3040 non-null   object        
 6   amount_usd       2084 non-null   object        
 7   remarks          3044 non-null   object        
 8   date_date        3036 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 237.8+ KB


In [59]:
df.isna().sum()

,0
startup_name,0
industry,171
subvertical,936
city,180
investors,24
InvestmentnType,4
amount_usd,960
remarks,0
date_date,8


In [60]:
px.bar(
    df.isna().sum() / 3044, # percent of missing values - to discard column higher than 30% missing values
    title='Missing Values',
    )

In [62]:
## convert amount_usd datatype to float

df.amount_usd = df.amount_usd.str.replace(',', '')

# astype(float)

In [63]:
df.amount_usd = pd.to_numeric(df.amount_usd, errors='coerce')

In [65]:
px.histogram(df.amount_usd, x='amount_usd', title='Amount in USD') #, log_y=True)

In [69]:
## clean city values

df.city.unique()

array(['Bengaluru', 'Gurgaon', 'New Delhi', 'Mumbai', 'Chennai', 'Pune',
       'Noida', 'Faridabad', 'San Francisco', 'San Jose,', 'Amritsar',
       'Delhi', 'Kormangala', 'Tulangan', 'Hyderabad', 'Burnsville',
       'Menlo Park', 'Gurugram', 'Palo Alto', 'Santa Monica', 'Singapore',
       'Taramani', 'Andheri', 'Chembur', 'Nairobi', 'Haryana', 'New York',
       'Karnataka', 'Mumbai/Bengaluru', 'Bhopal',
       'Bengaluru and Gurugram', 'India/Singapore', 'Jaipur', 'India/US',
       'Nagpur', 'Indore', 'New York, Bengaluru', 'California', 'India',
       'Ahemadabad', 'Rourkela', 'Srinagar', 'Bhubneswar', 'Chandigarh',
       'Delhi & Cambridge', 'Kolkatta', 'Kolkata', 'Coimbatore',
       'Bangalore', 'Udaipur', nan, 'Ahemdabad', 'Bhubaneswar',
       'Ahmedabad', 'Surat', 'Goa', 'Uttar Pradesh', 'Nw Delhi', 'Gaya',
       'Vadodara', 'Trivandrum', 'Missourie', 'Panaji', 'Gwalior',
       'Karur', 'Udupi', 'Kochi', 'Agra', 'Bangalore/ Bangkok', 'Hubli',
       'Kerala', 'Kozhiko

In [70]:
import pandas as pd
import numpy as np
import re

# Example: converting your array to a DataFrame column
cities = pd.Series([
    'Bengaluru', 'Gurgaon', 'San Jose,', '\\xc2\\xa0Noida', 'Ahemadabad', 'Kolkatta', np.nan
])

# Step 1: Remove leading/trailing whitespace & non-breaking spaces
def clean_whitespace(name):
    if pd.isna(name):
        return np.nan
    return re.sub(r'\s+', ' ', name.replace('\xa0', ' ')).strip().strip(',')

cities = cities.apply(clean_whitespace)

# Step 2: Normalize known typos and variants
replacements = {
    'Bangalore': 'Bengaluru',
    'Bangalore/ Bangkok': 'Bengaluru',
    'Kolkatta': 'Kolkata',
    'Ahemdabad': 'Ahmedabad',
    'Ahemadabad': 'Ahmedabad',
    'Gurugram': 'Gurgaon',
    'Nw Delhi': 'New Delhi',
    'Bhubneswar': 'Bhubaneswar',
    'Missourie': 'Mussoorie'
}

cities = cities.replace(replacements)

# Step 3: (Optional) Keep only the first city if multiple are listed
def first_city(name):
    if pd.isna(name):
        return np.nan
    return re.split(r'/|,| and | & ', name)[0].strip()

cities = cities.apply(first_city)

print(cities.unique())


['Bengaluru' 'Gurgaon' 'San Jose' '\\xc2\\xa0Noida' 'Ahmedabad' 'Kolkata'
 nan]


## Exploration

In [36]:
df.head()

,startup_name,industry,subvertical,city,investors,InvestmentnType,amount_usd,remarks,date_date
Sr No,,,,,,,,,
1,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",No Remarks,2020-01-09
2,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",No Remarks,2020-01-13
3,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",No Remarks,2020-01-09
4,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",No Remarks,2020-01-02
5,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",No Remarks,2020-01-02


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3044 entries, 1 to 3044
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   startup_name     3044 non-null   object        
 1   industry         2873 non-null   object        
 2   subvertical      2108 non-null   object        
 3   city             2864 non-null   object        
 4   investors        3020 non-null   object        
 5   InvestmentnType  3040 non-null   object        
 6   amount_usd       1 non-null      float64       
 7   remarks          3044 non-null   object        
 8   date_date        3036 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 237.8+ KB


In [68]:
df.groupby('city')['amount_usd'].sum().sort_values(ascending=False)

,amount_usd
city,
Bangalore,1.136159e+10
Bengaluru,7.098579e+09
Mumbai,4.921185e+09
New Delhi,3.017817e+09
Gurgaon,3.005296e+09
...,...
\\xc2\\xa0Bangalore,0.000000e+00
\\xc2\\xa0Gurgaon,0.000000e+00
\\xc2\\xa0Mumbai,0.000000e+00
